In [1]:
# https://huggingface.co/bond005/wav2vec2-mbart50-ru

In [8]:
import os
import warnings

import torch
from datasets import load_dataset, load_from_disk
from datasets.features import Audio
from transformers import SpeechEncoderDecoderModel, Wav2Vec2Processor

In [3]:
LANG_ID = "ru"
MODEL_ID = "bond005/wav2vec2-mbart50-ru"
PATH_MODEL = '/home/redalexdad/recognition_speech/wav2vec2-mbart50-ru'
# Кол-во текстов для предсказания
SAMPLES = 10

In [4]:
num_processes = max(1, os.cpu_count())

In [5]:
%%time
# Проверка наличия модели в локальном пути
if os.path.exists(PATH_MODEL):
    # Загрузка процессора из локального пути
    processor = Wav2Vec2Processor.from_pretrained(PATH_MODEL)
    
    # Загрузка модели из локального пути
    model = SpeechEncoderDecoderModel.from_pretrained(PATH_MODEL)
    print('Успешно модель загружена')
else:
    # Загрузка процессора из сети и сохранение в локальный путь
    processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
    processor.save_pretrained(PATH_MODEL)
    
    # Загрузка модели из сети и сохранение в локальный путь
    model = SpeechEncoderDecoderModel.from_pretrained(MODEL_ID)
    model.save_pretrained(PATH_MODEL)
    print(f'Успешно модель скачана и сохранена в пути {PATH_MODEL}')

Успешно модель загружена
CPU times: user 13.4 s, sys: 4.23 s, total: 17.6 s
Wall time: 30.6 s


In [11]:
%%time
test_dataset = load_dataset("mozilla-foundation/common_voice_11_0", LANG_ID, split=f"test[:{SAMPLES}]", trust_remote_code=True)

CPU times: user 372 ms, sys: 23.9 ms, total: 396 ms
Wall time: 3.95 s


In [12]:
if test_dataset.features['audio'].sampling_rate != 16_000:
    test_dataset = test_dataset.cast_column(
        'audio',
        Audio(sampling_rate=16_000)
    )

In [13]:
%%time
audio_data = [test_dataset[i]['audio']['array'] for i in range(SAMPLES)]

processed = processor(audio_data, sampling_rate=16_000, return_tensors="pt", padding='longest')

2024-03-08 20:51:12.286218: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 20:51:12.286342: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 20:51:12.478110: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-08 20:51:12.861734: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-08 20:51:14.253564: W tensorflow/compiler/tf2

In [14]:
%%time
with torch.no_grad():
    predicted_ids = model.generate(**processed)

In [15]:
predicted_sentences = processor.batch_decode(
    predicted_ids,
    num_processes=num_processes,
    skip_special_tokens=True
)

In [16]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for i, predicted_sentence in enumerate(predicted_sentences):
        print("-" * 100)
        print("Reference: ", test_dataset[i]["sentence"])
        print("Prediction:", predicted_sentence)

----------------------------------------------------------------------------------------------------
Reference:  К сожалению, эти предложения не нашли отражения в тексте.
Prediction: К сожалению, эти предложения не нашли отражения в тексте.
----------------------------------------------------------------------------------------------------
Reference:  Если не будет возражений, я буду считать, что Ассамблея согласна с этим предложением.
Prediction: Если не будет возражений, я буду считать, что Ассамблея согласна с этим предложением.
----------------------------------------------------------------------------------------------------
Reference:  Новошахтинск — милый город
Prediction: Новошахтинске милый город.
----------------------------------------------------------------------------------------------------
Reference:  Мы особенно рады отметить, что число скрывающихся от правосудия лиц уменьшилось.
Prediction: Мы все равно рады отметить, что число, в том числе в Таиланде, по совещанию Д